# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [2]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn.yml
    !mamba env update -n base -f environment_amn.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    if Stats == None:
        print('Stats for %s failed CPU-time %.4f' % (filename, time))
        return
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['README.md', 'Duplicate_Model.ipynb', 'Build_Model_Dense.ipynb', 'Build_Dataset.py', 'Dataset_experimental', '.ipynb_checkpoints', '.git', 'Build_Experimental.ipynb', 'Reservoir', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'Duplicate_Model.py', 'LICENSE', 'Build_Dataset.ipynb', 'Dataset_input', '__pycache__', 'Build_Experimental.py', 'old', 'environment_amn.yml', 'Build_Model.py', 'Build_Model.ipynb', '.DS_Store']


# Create and Train ANN




In [ ]:
# Create, train and evaluate ANN models with FBA simulated training set for E. coli core

# What you can change
seed = 10
np.random.seed(seed=seed)
trainname = 'e_coli_core_UB_10000'
size = 1000
# End of What you can change

# Load training set
trainingfile = DIRECTORY+'Dataset_model/'+trainname
cobramodel = cobra.io.read_sbml_model(trainingfile+'.xml')
OBJ = [r.id for r in cobramodel.reactions]
parameter = TrainingSet()
parameter.load(trainingfile)

# Set all pred fluxes to zero
Y, zeros = {}, 0 * parameter.Y 
for i in range(len(OBJ)):
    Y[i] = zeros[:,0] # zero vector

# train for different flux outputs 
# when size > len(OBJ) all flux are taken one after another 
# otherwise (small trainig sets) output fluxes are chosen at random
size = size if size < len(OBJ) else len(OBJ)
for k in range(size):
    
    # choose an output flux at random when size < range(len(OBJ))
    i = k if size == len(OBJ) else np.random.randint(0,high=len(OBJ))
    objective = [OBJ[i]]
    
    # create model
    print(k+1,'/',size, objective,'----------------------------------------------------------------------')
    if np.sum(parameter.Y[:,i]) == 0:
        continue # Skip all parameter.Y beeing at zero
    model = Neural_Model(trainingfile = trainingfile, 
                         objective=objective, 
                         model_type = 'ANN_Dense',
                         n_hidden = 1, hidden_dim = 50, 
                         epochs = 500, xfold = 5)
        
    # Train and evaluate
    start_time = time.time()
    try:
        reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
    except:
        reservoir, pred, stats, _ = None, zeros, None, None
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(objective, stats, reservoir, delta_time)
    Y[i] = pred[:,0]

# Collate all predicted Y and get stats and constraints
Y = np.transpose(np.asarray(list(Y.values())))
print(Y.shape, parameter.Y.shape)
print('Q2=', r2_score(parameter.Y, Y, multioutput='variance_weighted'))
X = tf.convert_to_tensor(np.float32(model.X)) # Loss computed of tf tensors
Y = tf.convert_to_tensor(np.float32(Y))
L2, _ = Loss_SV(Y, model.S)
L2 = np.mean(L2.numpy())
print('Loss_SV =', L2)
L3, _ = Loss_Vin(Y, model.Pin, X, 'UB')
L3 = np.mean(L3.numpy())
print('Loss_Vin =', L3)
L = (L2+L3)/2
print('Constraints =', L)

1 / 154 ['PFK'] ----------------------------------------------------------------------
nbr parameters: 1101
